"Group by" ismi SQL dilinden gelen bir komuttur. Ne olduğunu iyi açıklayan söz ise "böl, uygula, birleştir"dir.

### Böl, Uygula, Birleştir

`groupby` üç bölümden oluşur:

- İlk bölümde belirttiğimiz anahtar sütuna göre veriyi böleriz.
- Sonra böldüğümüz parçalar üzerinde istediğimiz işlemleri yaparız.
- En son işlem sonuçlarını birleştirerek sonuç elde ederiz.

Örnek verecek olursak en basit haliyle DataFrame metodu olarak `groupby()` kullanıp parametreye anahtar olarak kullanacağımız sütun adını verebiliriz.

In [1]:
import numpy as np
import pandas as pd

df = pd.DataFrame({
    'key': ['A', 'B', 'C', 'A', 'B', 'C'],
    'data1': range(6),
    'data2': np.random.randint(10, size=6)},
    columns=['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,1
1,B,1,4
2,C,2,5
3,A,3,3
4,B,4,1
5,C,5,1


In [2]:
df.groupby('key')

Görüldüğü üzere bir DataFrame değil, DataFrameGroupBy nesnesi döndürdü. Üzerinde işlem yapılana kadar o bir DataFrame değil. Bu nesne üzerinde aggregation ve birazdan göreceğimiz birtakım işlemleri yapabilirsiniz.

In [3]:
df.groupby('key').sum()

,data1,data2
key,,
A,3,4
B,5,5
C,7,6


### Groupby Nesnesi

Groupby ile çeşitli yollarla gruplama yapılabilir.

1. Sütun indexleme

`df.groupby('key')['data1']` şeklinde sütun üzerinde işlem yapabilirsiniz.

2. Döngülerle kullanma

GroupBy nesnesi iterasyon ile çalışmayı destekler, her grubu Serie veya DataFrame türünde verir. Yani `for` gibi döngülerle gruplar üzerinde çalışabilirsiniz.

3. Metod kullanımı

DataFrame ve Serie metodları GroupBy nesneleri ile kullanılabilir.

In [4]:
df.groupby('key')['data1'].describe()

,count,mean,std,min,25%,50%,75%,max
key,,,,,,,,
A,2.0,1.5,2.12132,0.0,0.75,1.5,2.25,3.0
B,2.0,2.5,2.12132,1.0,1.75,2.5,3.25,4.0
C,2.0,3.5,2.12132,2.0,2.75,3.5,4.25,5.0


### Aggregate, filter, transform, apply

GroupBy ile yapılan ana işlemler şunlardır: aggregate, filter, transform ve apply.

Bir önceki bölüm groupby ile verilerin gruplanıp, işlenip, birleştirilmesini gördük. Ama groupby ile yapabilecekleriniz bunlar ile sınırlı değil. Groupby nesnesi; gruplanmış verilere birleştirilmeden önce işlem yapabilmenizi sağlayan aggregate(), filter(), transform() ve apply() metodlarına da sahiptir.

##### Aggregate

`min()`, `sum()` gibi aggregate metodlarını görmüştük. `aggregate()` metoduna bunların esnek hali diyebiliriz. Parametre olarak fonksiyonun adı, fonksiyon veya bunlardan oluşan bir liste alabilir.

In [5]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     1    2.0   3
B       1    2.5   4     1    2.5   4
C       2    3.5   5     1    3.0   5

Kullanışlı başka bir örnek verirsek sütun adı ve karşısında ona uygulanacak işlemin yer aldığı sözlük verebiliriz.

In [6]:
df.groupby('key').aggregate({'data1': 'min', 'data2': 'max'})

,data1,data2
key,,
A,0,3
B,1,4
C,2,5


#### Filter

`filter()` metodu ile filtreleme yapabiliriz. Örneğe üç çiçek türünün yaprak uzunluklarını içeren ünlü *iris* veriseti üzerinden bakalım.

In [7]:
iris = pd.read_csv("../Data/Iris.csv")
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [8]:
iris.groupby("Species").aggregate("mean")

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
Species,,,,,
Iris-setosa,25.5,5.006,3.418,1.464,0.244
Iris-versicolor,75.5,5.936,2.770,4.260,1.326
Iris-virginica,125.5,6.588,2.974,5.552,2.026


Örnek verimizde üç türün de yapraklarıyla ilgili verilerin ortalamasını böyle görebiliyoruz. Aşağıda `filter` fonksiyonu ile taç yaprağı (PetalLengthCm) ortalama uzunluğu belli bir değerin üstünde olanları alıyoruz.

In [9]:
def filt(data):
    return 2 < data["PetalLengthCm"].min()

iris.groupby("Species").filter(filt)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
50,51,7.0,3.2,4.7,1.4,Iris-versicolor
51,52,6.4,3.2,4.5,1.5,Iris-versicolor
52,53,6.9,3.1,4.9,1.5,Iris-versicolor
53,54,5.5,2.3,4.0,1.3,Iris-versicolor
54,55,6.5,2.8,4.6,1.5,Iris-versicolor
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


Sonuç olarak *Iris-setosa*nın taç yaprağı ortalam uzunluğu 2'den küçük olduğundan eleniyor ve diğer iki türün bulunduğu veri dönüyor.

#### Transform

Transform gruplanmış veride veriler üzerinde düzenleme yapmayı sağlar. Aşağıda sık kullanılan bir örnek olan veriyi ortalama yapıyoruz.

In [10]:
centered_iris = iris.groupby('Species').transform(lambda x: x - x.mean())
centered_iris

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,-24.5,0.094,0.082,-0.064,-0.044
1,-23.5,-0.106,-0.418,-0.064,-0.044
2,-22.5,-0.306,-0.218,-0.164,-0.044
3,-21.5,-0.406,-0.318,0.036,-0.044
4,-20.5,-0.006,0.182,-0.064,-0.044
...,...,...,...,...,...
145,20.5,0.112,0.026,-0.352,0.274
146,21.5,-0.288,-0.474,-0.552,-0.126
147,22.5,-0.088,0.026,-0.352,-0.026
148,23.5,-0.388,0.426,-0.152,0.274


In [11]:
centered_iris["SepalLengthCm"].mean() # sütunların ortalaması neredeyse sıfır oldu

8.171241461241152e-16

#### Apply

Apply metodu gruplanmış veriler üzerinde düzenleme yapmayı sağlar. Transform'dan farklı olarak transform veri değerleri üzerinde değişiklik yaparken apply veriyi gruplayarak fonksiyona verir. Bu sayede sütunlar üzerinde çalışılabilir, verilen veriden farklı uzunluk ve boyutlarda veri döndürülebilir.

In [12]:
def fun(x):
    x['data1'] /= x['data2'].sum()
    return x

df.groupby('key').apply(fun)

,key,data1,data2
0,A,0.000000,1
1,B,0.200000,4
2,C,0.333333,5
3,A,0.750000,3
4,B,0.800000,1
5,C,0.833333,1


### Gruplama Kurallarını Belirleme

Şimdiye kadarki bölümde sütun adlarına göre gruplamayı öğrendik. Ama gruplamak için çok daha çeşitli yöntemler var.

1. Gruplamayı belirten liste: DataFrame'iniz ile aynı uzunluğa sahip bir liste (dizi, list, serie...) ile yapmak istediğiniz gruplamayı belirtebilirsiniz.

In [13]:
L = [0, 1, 0, 1, 2, 0]
df

,key,data1,data2
0,A,0,1
1,B,1,4
2,C,2,5
3,A,3,3
4,B,4,1
5,C,5,1


In [14]:
df.groupby(L).sum()

,data1,data2
0,7,7
1,4,7
2,4,1


2. Indexlerin gruplanmasını belirten sözlük: Indexlerin karşısına bulunacakları grupları belirten sözlük ile gruplama yapabilirsiniz.

In [15]:
df2 = df.set_index('key') # sütunu index yapıyoruz
df2

,data1,data2
key,,
A,0,1
B,1,4
C,2,5
A,3,3
B,4,1
C,5,1


In [16]:
mapping = {'A': 'A takımı', 'B': 'Diğer takım', 'C': 'Diğer takım'}

In [17]:
df2.groupby(mapping).sum()

,data1,data2
A takımı,3,4
Diğer takım,12,11


3. Fonksiyon: Indexin karşısında o indexin yer alacağı grubu belirten sözlük gibi,  parametre olarak veriyi verdiğinizde bulunacağı grubu verecek olan bir fonsiyon da verebilirsiniz.

In [18]:
df2.groupby(str.lower).sum()

,data1,data2
a,3,4
b,5,5
c,7,6


4. Bu gruplamaları birleştirme: Öğrendiğimiz özel gruplama kurallarını liste halinde vererek belirttiğiniz kurallara uygun gruplanmış MultiIndex yapısında sonucumuzu alabiliriz.

In [19]:
df2.groupby([str.lower, mapping]).mean()

,,data1,data2
a,A takımı,1.5,2.0
b,Diğer takım,2.5,2.5
c,Diğer takım,3.5,3.0
